# Inference
Get predictions for submission

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pretrainedmodels
import pandas as pd
import os

from tools import *


In [2]:
device = get_device()

Setting DEVICE:
	 MPS is available


## Importing label dataset

To run the full crossvalidation, remove .head(30000). This will multiply about by 20 the time required by this notebook.

In [3]:

# FOLDERS PATH
source_dir = 'histopathologic-cancer-detection/'
test_im_source_dir = source_dir+'/test'

# IMPORTING DATA
test_data = pd.read_csv(os.path.join(source_dir,'sample_submission.csv')) 



In [4]:
data_ids = list(test_data.id)
#data_labels = list(test_data.label)

## Loading model

In [5]:
model_source_dir = 'trained_model/'
model_name = 'final'

model_path =  model_source_dir+'/{}.model.pt'.format(model_name)


In [6]:
# Import pretrained model
base_model = pretrainedmodels.resnet34(num_classes=1000, 
                                    pretrained='imagenet').to(device) 
    
# Shape the model    
model = Net(base_model, 512).to(device)

# Load model from saved 
model.load_state_dict(torch.load(model_path))



/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

## Inference

In [7]:
test_loader = torch.utils.data.DataLoader(DataGeneratorInference(
                                                data_ids, 
                                                test_im_source_dir), 
                                            shuffle=False, 
                                            pin_memory=False, 
                                            num_workers=1,
                                            batch_size=1) 

model.eval() 

predictions = []
batch_prediction = []

with torch.no_grad():
    for batch_idx, x in enumerate(test_loader):
        # Test time augmentation

        

        image = np.rollaxis(x.numpy()[0], 0, 3)

        print(image.shape)

        images = produce_test_time_augmentation(image,number=8)
        
        output = model(torch.from_numpy(images).to(device, dtype=torch.float))
        output = output.mean()            
        #output = protein_model(x.to(device, dtype=torch.float))
        batch_prediction[batch_idx] = output
    
    predictions.append(batch_prediction)    
    
    

predictions = np.vstack(predictions) # get together the batches (one batch contains tta of one image)
predictions = np.mean(predictions, axis=0) # mean of tta of one image


test_data['label'] = predictions

test_data.to_cvs('predictions.csv', sep =',', index = False)

/Users/gabrielegabrielli/Documents/NN/Kaggle_CancerDetection/tools.py:320: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image =  torch.tensor(image, dtype = torch.float32)


torch.Size([1, 3, 96, 96])


TypeError: image must be numpy array type